## 什么是异常
异常是指程序语法正确，可以被执行，但是在执行过程中遇到了错误，即抛出异常。例子如下

In [1]:
# ZeroDivisionError
10 / 0

ZeroDivisionError: division by zero

In [2]:
# NameError
order * 2

NameError: name 'order' is not defined

In [3]:
# TypeError
1 + [1, 2]

TypeError: unsupported operand type(s) for +: 'int' and 'list'

## 如何处理异常

因为程序出现异常就会终止，在很多情景下我们并不想终止程序而是保持程序的运行，及时出现异常我们也应该通过某种方法去保证程序继续运行。
这就是所谓的异常处理，通过try和except来实现。

try会先尝试执行try block中的代码，如果出现异常且与except所要catch的异常类型相匹配，则会进入对应的except block中执行异常处理。except block 只接受与它相匹配的异常类型并执行，如果程序抛出的异常并不匹配，那么程序照样会终止并退出。

In [6]:
try:
    s = input('please enter two numbers separated by comma: ')
    num1 = int(s.split(',')[0].strip())
    num2 = int(s.split(',')[1].strip())
except ValueError as err:
    print('Value Error: {}'.format(err))

print('continue')

please enter two numbers separated by comma: 1


IndexError: list index out of range

## 如何处理多种异常
上述写法只能处理一种异常，当遇到多种异常时，仍然会终止执行。
处理多种异常时可以在except block中加入多种异常：

In [8]:
try:
    s = input('please enter two numbers separated by comma: ')
    num1 = int(s.split(',')[0].strip())
    num2 = int(s.split(',')[1].strip())

# 第一种写法
except (ValueError, IndexError) as err:
    print('Error: {}'.format(err))
    
# 第二种写法
# except ValueError as err:
#     print('Value Error: {}'.format(err))
# except IndexError as err:
#     print('Index Error: {}'.format(err))

print('continue')

please enter two numbers separated by comma: 1 
Error: list index out of range
continue


但是即便如此，也难保证程序会覆盖所有异常类型，因此，可以再except block的最后加入一个捕获非系统异常基类的block，保证覆盖全部非系统异常异常：

In [9]:
try:
    s = input('please enter two numbers separated by comma: ')
    num1 = int(s.split(',')[0].strip())
    num2 = int(s.split(',')[1].strip())
    
except ValueError as err:
    print('Value Error: {}'.format(err))
except IndexError as err:
    print('Index Error: {}'.format(err))
except Exception as err:
    print('Other error: {}'.format(err))

print('continue')

please enter two numbers separated by comma:    
Value Error: invalid literal for int() with base 10: ''
continue


如果想覆盖全部异常（包括系统异常或非系统异常），可以在except后面省略异常类型：

In [10]:
try:
    s = input('please enter two numbers separated by comma: ')
    num1 = int(s.split(',')[0].strip())
    num2 = int(s.split(',')[1].strip())
    
except ValueError as err:
    print('Value Error: {}'.format(err))
except IndexError as err:
    print('Index Error: {}'.format(err))
except:
    print('Other error')

print('continue')

please enter two numbers separated by comma: 
Value Error: invalid literal for int() with base 10: ''
continue


## 异常处理的收官
异常处理中，还有一个很常见的用法是 finally，经常和 try、except 放在一起来用。**在 finally 中，我们通常会放一些无论如何都要执行的语句**。无论发生什么情况，finally block 中的语句都会被执行，哪怕前面的 try 和 excep block 中使用了 return 语句。 一个比较常见的场景就是文件的读取，为了确保文件的完整性，无论读取成功还是失败都要关闭文件流。

In [16]:
import sys
try:
    f = open('file.txt', 'r')
    ... # some data processing
except OSError as err:
    print('OS error: {}'.format(err))
except:
    print('Unexpected error:', sys.exc_info()[0])
finally:
    f.close()

## 自定义异常类型
除了Python内置的异常类型，我们也可以自定义异常类型，自定义异常类型同内置异常类型一样继承自Exception类。

In [17]:
class MyInputError(Exception):
    """Exception raised when there're errors in input"""
    def __init__(self, value): # 自定义异常类型的初始化
        self.value = value
    def __str__(self): # 自定义异常类型的string表达形式
        return ("{} is invalid input".format(repr(self.value)))
    
try:
    raise MyInputError(1) # raise关键字抛出指定异常，抛出MyInputError这个异常
except MyInputError as err:
    print('error: {}'.format(err))

error: 1 is invalid input


## 异常的使用场景
通常来说，在程序中，如果我们不确定某段代码能否成功执行，往往这个地方就要使用异常处理。比如：文件读取、JSON的解码等。

大型社交网站的后台，需要针对用户发送的请求返回相应记录。用户记录往往储存在 key-value 结构的数据库中，每次有请求过来后，我们拿到用户的 ID，并用 ID 查询数据库中此人的记录，就能返回相应的结果。而数据库返回的原始数据，往往是 json string 的形式，这就需要我们首先对 json string 进行解码（json.loads()），但是json.loads()方法只能解码符合JSON规范的的字符串，否则就会抛出异常无法解码，所以解码JSON时候异常处理十分重要。

In [ ]:
try:
    data = json.loads(raw_data)
    ....
except JSONDecodeError as err:
    print('JSONDecodeError: {}'.format(err))

## 异常虽美但不可滥用
对于 flow-control（流程控制）的代码逻辑，我们一般不用异常处理。虽然在flow-control中使用异常处理代码并没有 bug，但是让人看了摸不着头脑，也显得很冗余。如果你的代码中充斥着这种写法，无疑对阅读、协作来说都是障碍。

In [18]:
# 不好推荐
# d = {'name': 'jason', 'age': 20}
# try:
#     value = d['dob']
#     ...
# except KeyError as err:
#     print('KeyError: {}'.format(err))

# 这样写就好
d = {'name': 'jason', 'age': 20}
if 'dob' in d:
    value = d['dob']
    ...